### Importing necessary modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import json
import requests
import os
import matplotlib.pyplot as plt
import cufflinks as cf
from functools import reduce
cf.set_config_file(world_readable=True,theme='pearl')
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

#Reading the data from csv and storing the df in empty dictionary
years = [2015,2016,2017,2018,2019]
df={}
for year in years:
    url = f"../UWA-Project/Resources/{year}.csv"
    pd.set_option("display.max_rows",800)
    df[year] = pd.read_csv(url)


### Cleaning the df over the period 2016-2019

In [2]:
#Unpacking the df
df_2015,df_2016,df_2017,df_2018,df_2019 = df[2015],df[2016],df[2017],df[2018],df[2019]

In [3]:
# Checking for null values in the df
for year in years:
    print(f"""{year} \n{df[year].isna().sum()}
----------""")

2015 
Country                          0
Region                           0
Happiness Rank                   0
Happiness Score                  0
Standard Error                   0
Economy (GDP per Capita)         0
Family                           0
Health (Life Expectancy)         0
Freedom                          0
Trust (Government Corruption)    0
Generosity                       0
Dystopia Residual                0
dtype: int64
----------
2016 
Country                          0
Region                           0
Happiness Rank                   0
Happiness Score                  0
Lower Confidence Interval        0
Upper Confidence Interval        0
Economy (GDP per Capita)         0
Family                           0
Health (Life Expectancy)         0
Freedom                          0
Trust (Government Corruption)    0
Generosity                       0
Dystopia Residual                0
dtype: int64
----------
2017 
Country                          0
Happiness.Rank          

In [4]:
# Checked for null values - replaced it with with zero.
df_2018.fillna(0,inplace = True)
df_2018.isna().sum()

Overall rank                    0
Country or region               0
Score                           0
GDP per capita                  0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
dtype: int64

In [5]:
# replacing spaces with underscore
for year in years:
    df[year].columns = [x.replace(" ","_") for x in df[year].columns.values]
    print(f"""{year} \n{df[year].columns}
----------""")


2015 
Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Standard_Error', 'Economy_(GDP_per_Capita)', 'Family',
       'Health_(Life_Expectancy)', 'Freedom', 'Trust_(Government_Corruption)',
       'Generosity', 'Dystopia_Residual'],
      dtype='object')
----------
2016 
Index(['Country', 'Region', 'Happiness_Rank', 'Happiness_Score',
       'Lower_Confidence_Interval', 'Upper_Confidence_Interval',
       'Economy_(GDP_per_Capita)', 'Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity',
       'Dystopia_Residual'],
      dtype='object')
----------
2017 
Index(['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high',
       'Whisker.low', 'Economy..GDP.per.Capita.', 'Family',
       'Health..Life.Expectancy.', 'Freedom', 'Generosity',
       'Trust..Government.Corruption.', 'Dystopia.Residual'],
      dtype='object')
----------
2018 
Index(['Overall_rank', 'Country_or_region', 'Score', 'GDP_per_capita',
      

In [6]:
#Removing non-contributing columns of the df 
col = ['Lower_Confidence_Interval','Upper_Confidence_Interval','Region','Whisker.high','Whisker.low','Standard_Error']
for year in years:
    for value in df[year].columns:
          if value in col:
            del df[year][value]

df[2019]

,Overall_rank,Country_or_region,Score,GDP_per_capita,Social_support,Healthy_life_expectancy,Freedom_to_make_life_choices,Generosity,Perceptions_of_corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
5,6,Switzerland,7.480,1.452,1.526,1.052,0.572,0.263,0.343
6,7,Sweden,7.343,1.387,1.487,1.009,0.574,0.267,0.373
7,8,New Zealand,7.307,1.303,1.557,1.026,0.585,0.330,0.380
8,9,Canada,7.278,1.365,1.505,1.039,0.584,0.285,0.308
9,10,Austria,7.246,1.376,1.475,1.016,0.532,0.244,0.226


In [7]:
#Checking the missing columns of the df
for year in years:
    print(f"""{year} \n{df[year].shape}
----------""")

2015 
(158, 10)
----------
2016 
(157, 10)
----------
2017 
(155, 10)
----------
2018 
(156, 9)
----------
2019 
(156, 9)
----------


In [8]:
#Calculating the missing dystopian columnn values for 2018 and 2019 and Renaming the columns over that period
for year in years[-2:]:
    df[year]['Dystopia_Residual'] = df[year].loc[:,'Score']-df[year].loc[:,'GDP_per_capita':].sum(axis=1)
    df[year].rename(columns={
    'Score':'Happiness_Score',
    'GDP_per_capita':'Economy_(GDP_per_Capita)',
    'Social_support':'Family',
    'Healthy_life_expectancy':'Health_(Life_Expectancy)',
    'Freedom_to_make_life_choices':'Freedom',
    'Perceptions_of_corruption':'Trust_(Government_Corruption)',
    'Overall_rank':'Happiness_Rank',
    'Country_or_region':'Country'
 },inplace=True)
    
#Renaming 2017 df column names
df[2017].rename(columns={
    'Score':'Happiness_Score',
    'GDP_per_capita':'Economy_(GDP_per_Capita)',
    'Social_support':'Family',
    'Healthy_life_expectancy':'Health_(Life_Expectancy)',
    'Freedom_to_make_life_choices':'Freedom',
    'Perceptions_of_corruption':'Trust_(Government_Corruption)',
    'Overall_rank':'Happiness_Rank',
    'Country_or_region':'Country'
 },inplace=True)


In [9]:
#Showing all the columns of the df are equal size
for year in years:
    print(f"""{year} \n{df[year].shape}
----------""")

2015 
(158, 10)
----------
2016 
(157, 10)
----------
2017 
(155, 10)
----------
2018 
(156, 10)
----------
2019 
(156, 10)
----------


In [23]:
#Finding any duplicates in df
for year in years:
    print(f"""{year} \n{df[year]['Country'].nunique()}
----------""")

2015 
158
----------
2016 
157
----------
2017 
155
----------
2018 
156
----------
2019 
156
----------


In [11]:
#Restructurin the column names for uniformity in df
#Writing the cleaned df into csv
for year in years:
    df[year] = df[year][['Country', 'Happiness_Rank', 'Happiness_Score',
       'Economy_(GDP_per_Capita)', 'Family', 'Health_(Life_Expectancy)',
       'Freedom', 'Trust_(Government_Corruption)', 'Generosity',
       'Dystopia_Residual']]
    df[year].to_csv(f"../UWA-Project/Output/{year}.csv",index=False)
    df[year].columns.values

In [13]:
data_frames = [df_2015,df_2016,df_2017,df_2018,df_2019]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Country'],how='outer'), data_frames).fillna(0)


In [14]:
df_merged

,Country,Happiness_Rank_x,Happiness_Score_x,Economy_(GDP_per_Capita)_x,Family_x,Health_(Life_Expectancy)_x,Freedom_x,Trust_(Government_Corruption)_x,Generosity_x,Dystopia_Residual_x,...,Dystopia_Residual_y,Happiness_Rank,Happiness_Score,Economy_(GDP_per_Capita),Family,Health_(Life_Expectancy),Freedom,Generosity,Trust_(Government_Corruption),Dystopia_Residual
0,Switzerland,1.0,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,...,2.318,6.0,7.480,1.452,1.526,1.052,0.572,0.263,0.343,2.272
1,Iceland,2.0,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,...,2.426,4.0,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2.401
2,Denmark,3.0,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,...,2.371,2.0,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2.394
3,Norway,4.0,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,...,2.383,3.0,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2.241
4,Canada,5.0,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,...,2.305,9.0,7.278,1.365,1.505,1.039,0.584,0.285,0.308,2.192
5,Finland,6.0,7.406,1.29025,1.31826,0.88911,0.64169,0.41372,0.23351,2.61955,...,2.585,1.0,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2.714
6,Netherlands,7.0,7.378,1.32944,1.28017,0.89284,0.61576,0.31814,0.47610,2.46570,...,2.448,5.0,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2.394
7,Sweden,8.0,7.364,1.33171,1.28907,0.91087,0.65980,0.43844,0.36262,2.37119,...,2.218,7.0,7.343,1.387,1.487,1.009,0.574,0.267,0.373,2.246
8,New Zealand,9.0,7.286,1.25018,1.31967,0.90837,0.63938,0.42922,0.47501,2.26425,...,2.156,8.0,7.307,1.303,1.557,1.026,0.585,0.330,0.380,2.126
9,Australia,10.0,7.284,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646,...,2.139,11.0,7.228,1.372,1.548,1.036,0.557,0.332,0.290,2.093
